# Ejercicio 6 fondos de inversion

- Consigue la rentabilidad los fondos de inversión en el año actual
- Recomendación: quefondos  https://www.quefondos.com/es/fondos/ranking/anual/index.html?cardCount=1&cardSize=100
- Tiempo objetivo: 45 minutos
- Objetivo extra: Programa la gestión de errores. Si da error al descargar, reinténtalo varias veces.

In [4]:
!pip install bs4
!pip install tqdm

You are using pip version 18.1, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [8]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

In [9]:
def obtener_fondos():
    
    url="https://www.quefondos.com/es/fondos/ranking/anual/index.html?cardCount=1&cardSize=100"
    soup  = requests.get(url)
    soup  = BeautifulSoup(soup.content, 'html.parser')
    name_box = soup.find('td', attrs={'colspan': '7'})
    
    name = name_box.text.strip() # strip() is used to remove starting and trailing
    name = name_box.text.split(' ')
    num_fondos = 1*float(name[len(name)-2])
    num_fondos = round(num_fondos/100,0)*100 # Redondeamos el número para que sea divisible entre 100.
    num_fondos_porc = num_fondos/100
    
    # Descargamos los fondos de inversión.
    for pagina in tqdm(range(int(num_fondos_porc))):
        url="https://www.quefondos.com/esfondos/ranking/anual/index.html?cardCount="+str(pagina+1)+"&cardSize=100"
        intentos=0
        descarga='sin datos'
        while descarga=='sin datos':
            try:
                page = requests.get(url)
                descarga = 'ok'
            except:
                intentos = intentos+1
                if intentos > 3:
                    print("Tras varios intentos no nos hemos podido descargar los fondos. Dejamos de intentarlo.")
                    break
                if descarga=="sin datos":
                    print("Error en la descarga de los fondos, reintentamos")
                    
        page = requests.get(url)
        table = soup.findAll('table')
        df = pd.read_html(str(table))[0]
        df = df.iloc[:,[0,1,4]] # nos quedamos con las columnas del nombre del fondo, la categoría y la rentabilidad
        df.columns = ['Nombre fondo','Categoria','Rentabilidad año en curso'] # Ponemos nombres a las columnas.
        n = df.shape[0]-1 # Eliminamos la fila resumen del final
        df = df.iloc[0:n,:]
        
        if pagina==0:
            dfacum=df
        else:
            dfacum=pd.concat([dfacum, df], axis=0)
            
        dfacum.reset_index(drop=True)
        
    return dfacum

In [10]:
tabla_fondos=obtener_fondos()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 355/355 [07:31<00:00,  1.27s/it]


In [11]:
tabla_fondos.head(10)

,Nombre fondo,Categoria,Rentabilidad año en curso
0,MULTIUNITS LUX - LYXOR S&P 500 VIX FUTURES ENH...,RVI USA,"147,00%"
1,"RENTA 4 ACTIVOS GLOBALES, FI I",MIXTO FLEXIBLE,"72,53%"
2,MSIF US INSIGHT Z (USD),RVI USA,"55,98%"
3,MSIF US INSIGHT A (USD),RVI USA,"55,31%"
4,MSIF US GROWTH Z (USD),RVI USA CRECIMIENTO,"55,21%"
5,MSIF US GROWTH I (USD),RVI USA CRECIMIENTO,"55,17%"
6,MSIF US GROWTH IX (USD),RVI USA CRECIMIENTO,"55,17%"
7,MSIF US GROWTH A (USD),RVI USA CRECIMIENTO,"54,62%"
8,MSIF US GROWTH AX (USD),RVI USA CRECIMIENTO,"54,61%"
9,MSIF US GROWTH IH (EUR),RVI USA CRECIMIENTO,"53,36%"


## Otra manera de hacerlo

In [ ]:
import time
import pandas as pd
from requests_html import HTMLSession

In [ ]:
def obtener_fondos():

    # Consultamos cuantos fondos están listados en la web.
    url = "https://www.quefondos.com/es/fondos/ranking/anual/index.html?cardCount=1&cardSize=100"
    datos_web = HTMLSession().get(url).html

    num_fondos = int(datos_web
                     .find('#report', first=True)
                     .find('.bold')[-1]  # ultimo bold
                     .text)

    # Redondeamos el número para que sea divisible entre 100.
    num_fondos = (round(num_fondos // 100, 0)) * 100

    # Descargamos los fondos de inversión.
    for pagina in range(1, (num_fondos // 100) + 1):

        url = f"https://www.quefondos.com/es/fondos/ranking/anual/index.html?cardCount={pagina}&cardSize=100"

        intentos = 0
        descarga = "Sin datos"

        while descarga == "Sin datos":

            try:
                datos_web = HTMLSession().get(url).html
                descarga = 'OK'
            except Exception:
                intentos = intentos + 1

                if intentos > 3:
                    print("Tras varios intentos no nos hemos podido descargar los fondos. Dejamos de intentarlo.")
                    return None

                if descarga == "Sin datos":
                    print("Error en la descarga de los fondos, reintentamos")
                    time.sleep(30)

        # Bajamos tabla entera de fondos
        fondos = pd.read_html(datos_web.find(".data_select", first=True).html)[0]
        fondos.columns = fondos.columns.droplevel()  # Borramos un nivel ya que incluye 2 iguales por defecto

        # Borramos última fila con "Resultados por página ..."
        fondos = fondos.iloc[:-1]

        if pagina == 1:
            fondos_acumulados = fondos
        else:
            fondos_acumulados = pd.concat([fondos_acumulados, fondos], ignore_index=True)

        print(f'{pagina * 100} fondos descargados de {num_fondos}')

    # seleccionamos columnas y las renombramos
    fondos_acumulados = fondos_acumulados.iloc[:, [0, 1, 4]]
    fondos_acumulados.columns = ["Nombre fondo", "Categoría", "Rentabilidad año en curso"]

    return(fondos_acumulados)

In [ ]:
tabla_fondos = obtener_fondos()

print(tabla_fondos

## Otra manera de hacerlo

In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
fondos=pd.DataFrame(columns=["Fondo", "Nombre", "Rentabilidad"])

url = 'https://www.quefondos.com/es/fondos/ranking/anual/index.html?cardCount=1&cardSize=100'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
texto=soup.find("tfoot")
sig=texto.find("tr")
siga=sig.findAll("span")
linea=str(siga[5])
linea=linea[linea.find("total"):]
linea=linea[(linea.find(">")+1):linea.find("</")]
num_fondos=int(linea)
num_fondos=int(round(num_fondos / 100, 0) * 100)
paginas=int(num_fondos/100)
    
    
for i in range(1, paginas):
    url = f'https://www.quefondos.com/es/fondos/ranking/anual/index.html?cardCount={i}&cardSize=100'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tabla=soup.find('table')
    trs=tabla.findAll("tr")    
    for tr in trs[3:]:
        tds = tr.findAll('td')
        a=str(tds[0])
        b=str(tds[1])
        c=str(tds[4])

        fondo=a[(a.find(":")+2):]
        fondo=fondo[:fondo.find('">')]
        nombre=b[(b.find(">")+1):(b.find("</"))]
        rentabilidad=c[(c.find(">")+1):c.find("</")]
        fondos.loc[fondos.shape[0]]=[fondo, nombre, rentabilidad]
        #print(f'{fondo} {nombre} {rentabilidad}')
    print(f'Hemos descargado {i*100} fondos')

Con la siguiente línea podemos ver 30 fondos de manera aleatoria.

In [ ]:
fondos.loc[np.random.choice(fondos.index,30)]

## Otra manera de hacerlo

In [ ]:
import requests
from lxml import html
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm
import pandas as pd
tqdm.pandas()

In [ ]:
itemsPerPage = 100
url = "https://www.quefondos.com/es/fondos/ranking/anual/index.html?cardCount=1&cardSize=100"
page = requests.get(url)
tree = html.fromstring(page.content)
nFondos = int(tree.xpath('//*[@id="report"]/span[3]')[0].text)
nFondos = int(np.floor(nFondos/itemsPerPage)*100)

In [ ]:
def getFunds(page:int, itemsPerPage:int):
    
    url = f"https://www.quefondos.com/es/fondos/ranking/anual/index.html?cardCount={page}&cardSize={itemsPerPage}"
    
    website_url = requests.get(url).text
    soup = BeautifulSoup(website_url)
    table = soup.find('table', attrs={'class':'data_select'})
    tableData = table.tbody.find_all("tr")

    tableDict = dict()
    for index, register in enumerate(tableData):
        tableDict[index] = dict()
        features = register.find_all("td")
        for feature in features:
            tableDict[index][feature['class'][0]] = feature.text
            
    return tableDict

In [ ]:
results = [getFunds(pagina+1, itemsPerPage) for pagina in tqdm(range(int(nFondos/itemsPerPage)))]

In [ ]:
dfResults = pd.DataFrame()
for batch in tqdm(results):
    local = pd.DataFrame.from_dict(batch, orient='index')
    dfResults = dfResults.append(local, ignore_index=True)
dfResults = dfResults[['fondo', 'np70', 'mas']].astype(str)
dfResults['mas'] = dfResults['mas'].progress_apply(lambda x: float(x.replace('%','').replace('.','').replace(',','.'))/100)
dfResults.columns = ["Nombre fondo","Categoría","Rentabilidad año en curso"]

In [ ]:
dfResults.head()